# NEURAL MACHINE TRANSLATION - Vanilla RNN

## Required Module & Config files

In [1]:
import evaluate
import numpy as np
from torch.nn import CrossEntropyLoss
from torch.optim import NAdam
from torchinfo import summary

import src.RNN_GRU as rnnNMT
from src.Tokenizer import Corpus, LangData, dataLoader
from src.Translator import Translator
from src.utils import load_config, get_device, train_model, sentence_bleu, corpus_bleu

# Loading config file
config = load_config()
# Get device : GPU/MPS Back-End/CPU
device = get_device()
print(f"Using device: {device}")

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


Using device: mps


## Load the dataset

In [2]:
# Encoder-Source
english_data = Corpus(f"{config.TRAIN_DATA}/english.txt", "English")
afrikaans_data = Corpus(f"{config.TRAIN_DATA}/afrikaans.txt", "Afrikaans")

## Set Hyperparameters

In [3]:
# Encoder - source
IN_ENCODER = english_data.vocab_size
ENCODER_EMB = 256

# Decoder - target
IN_DECODER = afrikaans_data.vocab_size
OUT_DECODER = afrikaans_data.vocab_size
DECODER_EMB = 256

# Shared
HIDDEN_SIZE = 1024
NUM_LAYERS = 2

LR = 1e-3
BATCH_SIZE = 128

## Set the model

In [4]:
encoder_net = rnnNMT.Encoder(IN_ENCODER, ENCODER_EMB, HIDDEN_SIZE, NUM_LAYERS).to(device)
decoder_net = rnnNMT.Decoder(IN_DECODER, DECODER_EMB, HIDDEN_SIZE, NUM_LAYERS).to(device)
model = rnnNMT.RNN_NMT(encoder_net, decoder_net, OUT_DECODER)

summary(model)

Layer (type:depth-idx)                   Param #
RNN_NMT                                  --
├─Encoder: 1-1                           --
│    └─RNN: 2-1                          3,411,968
│    └─Embedding: 2-2                    744,448
├─Decoder: 1-2                           --
│    └─RNN: 2-3                          3,411,968
│    └─Embedding: 2-4                    737,280
│    └─Linear: 2-5                       2,952,000
Total params: 11,257,664
Trainable params: 11,257,664
Non-trainable params: 0

In [5]:
train_data = LangData(english_data, afrikaans_data)
train_loader = dataLoader(train_data, BATCH_SIZE)

pad_idx = afrikaans_data.stoi['<pad>']
criterion = CrossEntropyLoss(ignore_index=0)

optimizer = NAdam(model.parameters(), LR)
translator = Translator(model, english_data, afrikaans_data, device)

In [6]:
# Data used for follow-up durring training
mytext = "<sos> given that we represent the target output as $y\in\{0,1\}$ and we have $n$ training points , we can write the negative log likelihood of the parameters as follows: <eos>"
ground = "<sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as: <eos>"

predicted = translator.translate_sentence(mytext)
bleu = sentence_bleu(prediction=[predicted], reference=[ground])
print(f"Pred: {predicted}")
print(f"Refe: {ground}")
print(f"BLEU SCORES: {bleu}")

Pred: <sos> maksimum selfoon moeilikste waai $y[-1]$ verkoue klub verkoue aandagtig ongemaklik laastens gemiddelde mari laat minimaliseer opgehou vurige spesifiek basismodel skrik normalisering teater teikenuittree waarskynlikheid drywingsseine punt werd verstaan beweeg $n=2048$ dwing $x[0],x[1],\ldots,x[19]$ gevoeg wolfgang honger 96\ toevoegingsmodel ""dankie"" is." ge\"implementeer hiberneer gebruik dir vroulik digtheid petrol gebore. overlap samelewing oorwin hebreeus regverdig belangstelle spele dieselfde toegelaat gisteraand verkeerd. hê egipte nee suster zero-gemiddelde halwe $\omega=\frac{\pi}{3}$ slaptjips opinie geelkoper loop muur mekaar derde niks $8$ b "lisa koning kleinstekwadraatpassing nuus gebeur \texttt{artist} bene vinnig stout gebou eeu teruggaan gesoek elkeen drankie vertel tyd-variant gooi bayes plaasgevind hoef straat regverdig gespasie\"{e}rde spreekbeurt gespasie\"{e}rde gevul huil vergelykings bewyse minimaliseer maklik petrol optree algebra\"ies kos bus gaand

## Train the data

In [7]:
EPOCHS = 50
params = {
    "model": model,
    "train_loader": train_loader,
    "optimizer": optimizer,
    "criterion": criterion,
    "device": device,
    "epochs": EPOCHS,
    "source_test": mytext,
    "reference": ground,
    "translator": translator
}

train_loss = train_model(**params)
np.save('rnn_train_loss.npy', np.array(train_loss))

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.
Epoch 1/50: 100%|██████████| 20/20 [00:03<00:00,  5.07batch/s, loss=1.730]


Predicted: <sos> ek is 'n die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die volgende van die
B

Epoch 2/50: 100%|██████████| 20/20 [00:03<00:00,  5.64batch/s, loss=1.576]


Predicted: <sos> ek is nie <eos>
BLEU Score: [0.024, 0.021, 0.016, 0.0]


Epoch 3/50: 100%|██████████| 20/20 [00:03<00:00,  5.57batch/s, loss=1.444]


...

Predicted: <sos> die sein $x(t)$ word geanaliseer deur 'n spektrogram te gebruik met die doel om die onmiddelike frekwensie elke 2 ms te bepaal , met 'n frekwensieresolusie van 200 hz en volle hooflob skeiding <eos>
BLEU Score: [0.273, 0.159, 0.107, 0.0]


Epoch 49/50: 100%|██████████| 20/20 [00:03<00:00,  5.58batch/s, loss=0.556]


Predicted: <sos> vir 'n vaste waarde van ik en matriek gemiddeld , verdien mans meer as vrouens in die gemiddeld <eos>
BLEU Score: [0.245, 0.151, 0.104, 0.0]


Epoch 50/50: 100%|██████████| 20/20 [00:03<00:00,  5.52batch/s, loss=0.575]

Predicted: <sos> as ons die data hier bo op hierdie manier verwerk , sal dit nie beteken dat vir elk van die drie klasse al die data rondom die oorsprong sal l\^{e} met eenheidsvariansie nie <eos>
BLEU Score: [0.318, 0.228, 0.183, 0.146]


## Evauate on the Training set

In [8]:
EN_SRC = [' '.join(sent) for sent in english_data.data_str]
AF_REF = [[' '.join(sent)] for sent in afrikaans_data.data_str]
TRANSLATED = [translator.translate_sentence(sent) for sent in EN_SRC]
corpus_bleu(TRANSLATED, AF_REF)

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


                                     BLEU-1                                     
------------------------------------------------------------------------------------------
bleu                : 0.16426953733933353
precisions          : [0.1642695373393335]
brevity_penalty     : 1.0
length_ratio        : 2.762798634812287
translation_length  : 101997
reference_length    : 36918
******************************************************************************************
                                     BLEU-2                                     
------------------------------------------------------------------------------------------
bleu                : 0.12267072962141941
precisions          : [0.1642695373393335, 0.09160619887037447]
brevity_penalty     : 1.0
length_ratio        : 2.762798634812287
translation_length  : 101997
reference_length    : 36918
******************************************************************************************
                                     

## Evaluate on the Test set

In [9]:
with open(f"{config.VAL_DATA}/english.txt") as data:
    english_val = data.read().strip().split("\n")
with open(f"{config.VAL_DATA}/afrikaans.txt") as data:
    afrikaans_val = data.read().strip().split("\n")

### Greedy Search

In [10]:
VAL_AF_REF = [[sent] for sent in afrikaans_val]
VAL_TRANSLATED = [translator.translate_sentence(sent) for sent in english_val]

corpus_bleu(VAL_TRANSLATED, VAL_AF_REF)

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


                                     BLEU-1                                     
------------------------------------------------------------------------------------------
bleu                : 0.1588040795417501
precisions          : [0.1588040795417501]
brevity_penalty     : 1.0
length_ratio        : 2.941103958361868
translation_length  : 42946
reference_length    : 14602
******************************************************************************************
                                     BLEU-2                                     
------------------------------------------------------------------------------------------
bleu                : 0.11942361867713319
precisions          : [0.1588040795417501, 0.08980878034806274]
brevity_penalty     : 1.0
length_ratio        : 2.941103958361868
translation_length  : 42946
reference_length    : 14602
******************************************************************************************
                                     BLE

### Beam search

In [11]:
VAL_TRANSLATED = [translator.translate_sentence(sent, method="beam", beam_width=3) for sent in english_val]
corpus_bleu(VAL_TRANSLATED, VAL_AF_REF)

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


                                     BLEU-1                                     
------------------------------------------------------------------------------------------
bleu                : 0.1588040795417501
precisions          : [0.1588040795417501]
brevity_penalty     : 1.0
length_ratio        : 2.941103958361868
translation_length  : 42946
reference_length    : 14602
******************************************************************************************
                                     BLEU-2                                     
------------------------------------------------------------------------------------------
bleu                : 0.11942361867713319
precisions          : [0.1588040795417501, 0.08980878034806274]
brevity_penalty     : 1.0
length_ratio        : 2.941103958361868
translation_length  : 42946
reference_length    : 14602
******************************************************************************************
                                     BLE

## Evaluate on the SUN validation set only

In [12]:
with open(f"{config.VAL_DATA}/sun_english.txt") as data:
    sun_english_val = data.read().strip().split("\n")
with open(f"{config.VAL_DATA}/sun_afrikaans.txt") as data:
    sun_afrikaans_val = data.read().strip().split("\n")

### Greedy Search

In [13]:
SUN_VAL_AF = [[sent] for sent in sun_afrikaans_val]
SUN_VAL_TRANSLATED = [translator.translate_sentence(sent) for sent in sun_english_val]
corpus_bleu(SUN_VAL_TRANSLATED, SUN_VAL_AF)

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


                                     BLEU-1                                     
------------------------------------------------------------------------------------------
bleu                : 0.2029998728867421
precisions          : [0.2029998728867421]
brevity_penalty     : 1.0
length_ratio        : 2.0626638699528055
translation_length  : 7867
reference_length    : 3814
******************************************************************************************
                                     BLEU-2                                     
------------------------------------------------------------------------------------------
bleu                : 0.14150133911657078
precisions          : [0.2029998728867421, 0.09863370201691607]
brevity_penalty     : 1.0
length_ratio        : 2.0626638699528055
translation_length  : 7867
reference_length    : 3814
******************************************************************************************
                                     BLEU-

### Beam Search

In [14]:
SUN_VAL_TRANSLATED = [translator.translate_sentence(sent, method="beam", beam_width=3) for sent in sun_english_val]
corpus_bleu(SUN_VAL_TRANSLATED, SUN_VAL_AF)

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


                                     BLEU-1                                     
------------------------------------------------------------------------------------------
bleu                : 0.20331687555386757
precisions          : [0.20331687555386757]
brevity_penalty     : 1.0
length_ratio        : 2.071054011536445
translation_length  : 7899
reference_length    : 3814
******************************************************************************************
                                     BLEU-2                                     
------------------------------------------------------------------------------------------
bleu                : 0.14206164793753753
precisions          : [0.20331687555386757, 0.09926137099909291]
brevity_penalty     : 1.0
length_ratio        : 2.071054011536445
translation_length  : 7899
reference_length    : 3814
******************************************************************************************
                                     BLEU

In [15]:
metric = evaluate.load("bleu")
predictions = [translator.translate_sentence(sent, method="beam", beam_width=5) for sent in sun_english_val[10:20]]
labels = SUN_VAL_AF[10:20]
for source, pred, lab in zip(sun_english_val[10:20],predictions, labels):
    print(f"Source    : {source}")
    print(f"Prediction: {pred[:150]}")
    print(f"Label     : {lab[0][:150]}")
    print(f"BLEU      : {metric.compute(predictions=[pred], references=lab)['bleu']}")
    print()

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


Source    : <sos> embedded computing <eos>
Prediction: <sos> as ons die data hier bo op hierdie manier verwerk , sal dit nie beteken dat vir elk van die drie klasse al die data rondom die oorsprong sal l\^
Label     : <sos> toegewyde verwerking <eos>
BLEU      : 0.0

Source    : <sos> uml description <eos>
Prediction: <sos> as ons die data hier bo op hierdie manier verwerk , sal dit nie beteken dat vir elk van die drie klasse al die data rondom die
Label     : <sos> uml beskrywing <eos>
BLEU      : 0.0

Source    : <sos> system design <eos>
Prediction: <sos> as ons die data hier bo op hierdie manier verwerk , sal dit nie beteken dat vir elk van die drie klasse al die data
Label     : <sos> stelsel ontwerp <eos>
BLEU      : 0.0

Source    : <sos> program design and analysis <eos>
Prediction: <sos> as ons die data hier bo op hierdie manier verwerk , sal dit nie beteken dat vir elk van die drie klasse al die data rondom die oorsprong sal l\^
Label     : <sos> program ontwerp en analise <e